# <span style="color:green">Formation South Green 2022</span> - Structural Variants Detection by using short and long reads 

# __DAY 2 : SNP calling__

Created by C. Tranchant (DIADE-IRD), J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and A. Dereeper (PHIM-IRD)

## __1. Preparing the working environment__ 

### First create a dedicated folder to work 

In [ ]:
# go to work directory
cd /home/jovyan/work/
ls

### We are going to use only one clones to check is all is ok before run the whole of samples! 

In [ ]:
## declare variables
i=10
REF_DIR="/home/jovyan/work/SV_DATA/REF/"
REF="/home/jovyan/work/SV_DATA/REF/reference.fasta"
ONT="/home/jovyan/work/SV_DATA/LONG_READS/Clone$i.fastq.gz"
ILL_R1="/home/jovyan/work/SV_DATA/SHORT_READS/Clone${i}_R1.fastq.gz"
ILL_R2="/home/jovyan/work/SV_DATA/SHORT_READS/Clone${i}_R2.fastq.gz"
BAM="/home/jovyan/work/MAPPING-ILL/dirClone10/Clone$i.SORTED.bam"

In [ ]:
# go to SR mapping results
cd /home/jovyan/work/MAPPING-ILL/dirClone${i}

In [ ]:
ls /home/jovyan/work/MAPPING-ILL/dirClone${i}

### Mark duplications

In [ ]:
echo -e "\nMarkDuplicates in Clone$i";
gatk MarkDuplicates -I $BAM -M duplicates.$i.metrics -O Clone$i.SORTED.MD.bam;
samtools index Clone$i.SORTED.MD.bam;

In [ ]:
# check MD bam
ls -l

### Indexing reference with GATK

In [ ]:
cd $REF_DIR
samtools faidx $REF
gatk CreateSequenceDictionary -R $REF

### SNP Calling for a Clone

In [ ]:
# go to bam repertory
cd /home/jovyan/work/MAPPING-ILL/dirClone${i}
# change BAM file for MD one
BAM="/home/jovyan/work/MAPPING-ILL/dirClone10/Clone$i.SORTED.MD.bam"
# lauch GATK HaplotypeCaller
echo -e "\nCalling Clone$i";
gatk --java-options "-Xmx4g" HaplotypeCaller --native-pair-hmm-threads 4 -I Clone$i.SORTED.MD.bam -O Clone$i.g.vcf -R $REF -ERC GVCF;

In [ ]:
head Clone$i.g.vcf

### RG is important !! 

Corrects bams files with `-R "@RG\tID:Clone${i}\tSM:Clone${i}"` are available in XXX.


In [ ]:
wget XXXXXXXXXXX
BAM_ILL="/home/jovyan/work/BAM_ILL"
BAM_ONT="/home/jovyan/work/BAM_ONT"

In [ ]:
ls $BAM_ILL

In [ ]:
ls $BAM_ONT

## Calling all samples on one raw VCF with correct BAM files

In [ ]:
# create a work directory to SNP calling results
mkdir /home/jovyan/work
cd /home/jovyan/work/VCF/VCF

In [ ]:
## GATK HC with all samples

In [ ]:
cd $BAM_ILL
for i in {1..20}
    do
        echo -e "\nMarkDuplicates in Clone$i";
        gatk MarkDuplicates -I Clone$i.SORTED.bam; -M duplicates.$i.metrics -O Clone$i.SORTED.MD.bam;
        samtools index Clone$i.SORTED.MD.bam;
    done

In [ ]:
# Loop to inflate the --variant option
OPTION=""
for i in {1..20}
do
    OPTION="${OPTION} --variant dirClone${i}/Clone${i}.g.vcf"
done
echo $OPTION
# GATK
gatk CombineGVCFs -R referenceCorrect.fasta $OPTION -O rawSNP.vcf

# Have a Look to it

In [ ]:
head -n 1000 rawSNP.vcf | tail

## Compute the Genotypes

In [ ]:
gatk --java-options "-Xmx4g" GenotypeGVCFs -R reference.fasta -V rawSNP.vcf -O output.vcf

## Compute the SNP density along the chromosomes

In [ ]:
echo -e "Reference\t1000000\n" > /ifb/data/mydatalocal/MappingAndSNP/genome.txt
bedtools genomecov -bga -split -i /ifb/data/mydatalocal/MappingAndSNP/output.vcf -g /ifb/data/mydatalocal/MappingAndSNP/genome.txt > /ifb/data/mydatalocal/MappingAndSNP/density.csv

In [ ]:
head /ifb/data/mydatalocal/MappingAndSNP/density.csv